In [1]:
import json
import re
import warnings

from spacy.tokens import Token, Span, Doc
from tornado.template import Template

from generator import utils, grammar
import numpy as np
import pandas as pd

from source import ElasticResponse,read_query
from generator.search import templatize
from generator.utils import load_spacy_model
nlp = load_spacy_model()

In [2]:
er = ElasticResponse()
sql_query = read_query("backup","policy_complaint","retention_status")
df = er.get_response(sql_query,{})

In [3]:
df

,Host Name,STATE,Count
0,MRS-IS-00021,COMPLIANT,1148
1,MRS-IS-00021,WARNING,2716
2,MRS-IS-00023,COMPLIANT,1176
3,MRS-IS-00023,WARNING,1876
4,RLTIA00007,COMPLIANT,1209
5,RLTIA00007,WARNING,2511
6,RLTIA00008,COMPLIANT,1054
7,RLTIA00008,WARNING,2511


In [17]:
df = df.pivot(index="Host Name",columns="STATE")
df.columns = [c[1] for c in df.columns]

In [18]:
df["CRITICAL"] = 0 if not "CRITICAL" in df.columns else df["CRITICAL"]

In [19]:
df = ((df.transpose()/ df.sum(axis=1)).transpose() * 100).round(2)
df.sort_values(["CRITICAL","WARNING"],ascending=False,inplace=True)

In [20]:
pd.set_option('display.width', 1000)
pd.set_option('colheader_justify', 'center')
df['Health'] = df.apply(lambda x: "Red" if x['CRITICAL']>0 else "Yellow",axis=1)
df.Health.fillna('Green',inplace=True)

In [21]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th></th>\n      <th>COMPLIANT</th>\n      <th>WARNING</th>\n      <th>CRITICAL</th>\n      <th>Health</th>\n    </tr>\n    <tr>\n      <th>Host Name</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>RLTIA00008</th>\n      <td>29.57</td>\n      <td>70.43</td>\n      <td>0.0</td>\n      <td>Yellow</td>\n    </tr>\n    <tr>\n      <th>MRS-IS-00021</th>\n      <td>29.71</td>\n      <td>70.29</td>\n      <td>0.0</td>\n      <td>Yellow</td>\n    </tr>\n    <tr>\n      <th>RLTIA00007</th>\n      <td>32.50</td>\n      <td>67.50</td>\n      <td>0.0</td>\n      <td>Yellow</td>\n    </tr>\n    <tr>\n      <th>MRS-IS-00023</th>\n      <td>38.53</td>\n      <td>61.47</td>\n      <td>0.0</td>\n      <td>Yellow</td>\n    </tr>\n  </tbody>\n</table>'

In [22]:
text = """<p><strong>Retention Policy Status:&nbsp;</strong></p>
        <p>&nbsp;</p>
        <p>This section describes the discrepancies in the backup retention policies created on the server.</p>
        <p>&nbsp;</p>
        <p><strong>Description</strong>: There are 3 different statuses (Warning, Critical, and Complaint).</p>
        <p>&nbsp;</p>
        <p>* <em>Critical</em>: Retention policy does not exist on the server but will be available in the document.</p>
        <p>* <em>Warning</em>: Retention policy exists on the server but will not be available in the document.</p>
        <p>* <em>Complaint</em>: It adheres to the policy.</p>
        <p>&nbsp;</p>"""
text = nlp(text)

In [23]:
nugget = templatize(text,{} , df)


In [27]:
html_content = nugget.render(df).decode("utf-8")  + df.drop('Health',axis=1).to_html() 
html_content += pd.DataFrame(df.groupby('Health').apply(lambda x: ', '.join(x.index)),columns=["Hosts"]).to_html()

In [29]:
from IPython.core.display import HTML
HTML(html_content)

In [30]:
html_content

'<p><strong>Retention Policy Status:&nbsp;</strong></p> <p>&nbsp;</p> <p>This section describes the discrepancies in the backup retention policies created on the server.</p> <p>&nbsp;</p> <p><strong>Description</strong>: There are 3 different statuses (Warning, Critical, and Complaint).</p> <p>&nbsp;</p> <p>* <em>Critical</em>: Retention policy does not exist on the server but will be available in the document.</p> <p>* <em>Warning</em>: Retention policy exists on the server but will not be available in the document.</p> <p>* <em>Complaint</em>: It adheres to the policy.</p> <p>&nbsp;</p><table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th></th>\n      <th>COMPLIANT</th>\n      <th>WARNING</th>\n      <th>CRITICAL</th>\n    </tr>\n    <tr>\n      <th>Host Name</th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>RLTIA00008</th>\n      <td>29.57</td>\n      <td>70.43</td>\n      <td>0.0